In [1]:
!mkdir results

mkdir: cannot create directory ‘results’: File exists


In [2]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

import pandas as pd

gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/12_IDpgV2_QsGX0IL8x5bCpNDkVbFELVJgW2va6oNWhs')

In [3]:
out_string = """
# Results

## japan asphalt rally (warmup event)

Congratulations to Wes, turbo and KK for rounding out the podium!

| rank | name              | min     | link                                         |
| ---- | ----------------- | ------- | -------------------------------------------- |
| 1.   | Wesley Daggleston | 08:46.7 | https://youtu.be/yrHIN4VkbfM                 |
| 2.   | turbo             | 08:50.3 | https://youtu.be/PXC8I5Oxz9U                 |
| 3.   | kk                | 08:51.6 | https://youtu.be/55p4QR5be0k                 |
| 4.   | Hulter            | 08:56.1 | https://youtu.be/-g5NBrjKzA8                 |
| 5.   | nept              | 08:58.5 | https://www.youtube.com/watch?v=en_yPK5yPG0 |
| 6.   | Red Sun           | 09:00.1 | https://www.twitch.tv/videos/1607608745      |
| 7.   | capablanca	   | 09:11.3 | https://youtu.be/yhjtPe59kZg
| 8.   | domer             | 09:12.8 | https://youtu.be/rBMY4pOHQu8                 |
| 9.   | MxCraven          | 09:43.8 | https://www.twitch.tv/videos/1607365265      |
|10.   | nap               | 09:50.5 | https://www.youtube.com/watch?v=OjfTEOvrbTY  |
"""

comments = [
    "Well done Proporo!",
    "",
    "",
    "",
]

In [4]:
rallies = ['indonesia', 'kenya', 'germany', 'sardinia']

In [5]:
points_ladder = [20,15,12,10,8,6,4,3,2,1]
ps_ladder = [4,3,2,1]

def rally_points(pos):
    if pos <= 10:
        return points_ladder[round(pos) - 1]
    else:
        return 0

def ps_points(pos):
    if pos <= 4:
        return ps_ladder[round(pos) - 1]
    else:
        return 0


In [6]:
sheets = gsheets.worksheet('teams').get_all_values()

team_df = pd.DataFrame(sheets[0:], columns=sheets[0]).query('team != " "')

team_df = team_df[['name', 'team']]

In [14]:
standings = 0
n=0

for rally in rallies:

    sheets = gsheets.worksheet(rally).get_all_values()
    df = pd.DataFrame(sheets[3:], columns=sheets[2]).query('rank != ""')

    results = df[['name', 'min time', 'power stage', 'link']]
    results.columns = ['name', 'time', 'powerstage', 'link']

    results = results.merge(team_df, on='name', how='left').fillna("")

    results['pos'] = results['time'].rank()
    results['pspos'] = results['powerstage'].rank()
    results['rally'] = results.pos.apply(rally_points)
    results['ps'] = results.pspos.apply(ps_points)
    results['points'] = (results.rally + results.ps).astype(int)

    results = results[['pos', 'name', 'time', 'powerstage', 'link', 'rally', 'ps', 'points', 'team']]   
    results.to_markdown('results/' + rally + '.md', index=False)

    out_string += '\n' + '## ' + rally
    out_string += '\n' + comments[n]
    out_string += '\n' + results.to_markdown(index=False)

    if type(standings) == int:
        standings = results[['name', 'points']].rename(columns={'points': rally})
    else:
        standings = standings.merge(
            results.rename(columns={'points': rally})[['name', rally]], 
            on='name',
            how='left'
        )
    
    n += 1


standings['total'] = standings[rallies].fillna(0).sum(axis=1) - standings[rallies].fillna(0).min(axis=1)

standings = standings.sort_values(by='total', ascending=False)

standings.to_markdown('results/' + 'standings.md', index=False)
out_string += '\n' + "## standings"
out_string += '\n' + standings.fillna('-').to_markdown(index=False)



# team standings

In [15]:
team_standings = standings.merge(team_df, on='name')

for r in rallies:
    team_standings["t_" + r] = team_standings.groupby("team")[r].rank(ascending=False, method='dense')

    team_standings["t2_" + r] = (
        team_standings["t_" + r]
        .apply(lambda x : 1 if x < 3 else 0) 
        * team_standings[r].apply(lambda x : None if x == 0 else x).rank(ascending=False)
    ).apply(rally_points)

team_standings = team_standings.groupby('team')[rallies + ['total']].sum().sort_values(by='total', ascending=False).reset_index()

print(team_standings)

                team  indonesia  kenya  germany  sardinia  total
0   cat face nospace         30    0.0      0.0       0.0   30.0
1         g2d racing         28    0.0      0.0       0.0   28.0
2  Brick Motorsports         16    0.0      0.0       0.0   16.0


In [16]:
team_standings.to_markdown('results/team_standings.md', index=False)

In [17]:
out_string += '\n' + "## team standings"
out_string += '\n' + team_standings.to_markdown(index=False)

In [18]:
print(out_string)


# Results

## japan asphalt rally (warmup event)

Congratulations to Wes, turbo and KK for rounding out the podium!

| rank | name              | min     | link                                         |
| ---- | ----------------- | ------- | -------------------------------------------- |
| 1.   | Wesley Daggleston | 08:46.7 | https://youtu.be/yrHIN4VkbfM                 |
| 2.   | turbo             | 08:50.3 | https://youtu.be/PXC8I5Oxz9U                 |
| 3.   | kk                | 08:51.6 | https://youtu.be/55p4QR5be0k                 |
| 4.   | Hulter            | 08:56.1 | https://youtu.be/-g5NBrjKzA8                 |
| 5.   | nept              | 08:58.5 | https://www.youtube.com/watch?v=en_yPK5yPG0 |
| 6.   | Red Sun           | 09:00.1 | https://www.twitch.tv/videos/1607608745      |
| 7.   | capablanca	   | 09:11.3 | https://youtu.be/yhjtPe59kZg
| 8.   | domer             | 09:12.8 | https://youtu.be/rBMY4pOHQu8                 |
| 9.   | MxCraven          | 09:43.8 | https:

# update sheet

In [19]:
stand_sheet = gsheets.worksheet('standings')

for y in range(standings.shape[1]):

    stand_sheet.update_cell(1, y+1, str(list(standings.columns)[y]))

    for x in range(standings.shape[0]):
        stand_sheet.update_cell(x+2, y+1, str(standings.iloc[x,y]))

In [20]:
team_stand = gsheets.worksheet('team_standings')

for y in range(team_standings.shape[1]):

    team_stand.update_cell(1, y+1, str(list(team_standings.columns)[y]))

    for x in range(team_standings.shape[0]):
        team_stand.update_cell(x+2, y+1, str(team_standings.iloc[x,y]))